<a href="https://colab.research.google.com/github/kovaciardey/ce888-assignment/blob/main/alex_net_flame_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ALexNet CNN training with the FLAME dataset 

This implementation of the AlexNet CNN has been created following a tutorial on [this](https://towardsdatascience.com/implementing-alexnet-cnn-architecture-using-tensorflow-2-0-and-keras-2113e090ad98) article on the TowardsDataScience website. Some tweaks have been made to the functionality of the workflow to suit the FLAME dataset.

In [ ]:
# importing the libraries

from numpy import load
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
import os
import time
import seaborn as sns

import pandas as pd

## !! IMPORTANT !!

It is recomended to have the data files generated by the data_splitting notebook at the same level in folder as this notebook.

Additionally, it should be mentioned that the code presented in this notebook had been executed locally from the PC and not on Google colab. It is my recommendation to do the same as there may be issues with loading the data. 

The python script has been converted into this notebook to enable the execution of the code in snippets.

In [ ]:
# loading the data and verifying the correct shape and size

set_size = 500

# the name of the 4 files to load. The path can be updated here to load the data from a different place.
train_images = load("training_flame_photos.npy")
train_labels = load("training_flame_labels.npy")

test_images = load("test_flame_photos.npy")
test_labels = load("test_flame_labels.npy")

CLASSES = ["No Fire", "Fire"]

validation_images, validation_labels = train_images[:set_size], train_labels[:set_size]
train_images, train_labels = train_images[set_size:], train_labels[set_size:]

train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))

print("  Training Data Shape:", train_ds)
print("      Test Data Shape:", test_ds)
print("Validation Data Shape:", validation_ds)

train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()
validation_ds_size = tf.data.experimental.cardinality(validation_ds).numpy()

print("  Training data size:", train_ds_size)
print("      Test data size:", test_ds_size)
print("Validation data size:", validation_ds_size)

In [ ]:
# processing the images into the correct size

def process_images(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    image = tf.image.per_image_standardization(image)
    # Resize images from 32x32 to 277x277
    image = tf.image.resize(image, (227, 227))
    return image, label

train_ds = (train_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))
test_ds = (test_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))
validation_ds = (validation_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))

print(train_ds)
print(test_ds)
print(validation_ds)


In [ ]:
# configuring the model
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001), metrics=['accuracy'])
model.summary()

In [ ]:
# training the model
model.fit(train_ds,
          epochs=50,
          validation_data=validation_ds,
          validation_freq=1,
          callbacks=[tensorboard_cb])

In [ ]:
# resizing the images in the original 

print(test_ds_size)

def process_images2(image):
    image = tf.image.per_image_standardization(image)
    image = tf.image.resize(image, (227, 227))
    return image

new_train_ds = tf.data.Dataset.from_tensor_slices((test_images, ))

new_train_ds = (new_train_ds.map(process_images2).batch(batch_size=32, drop_remainder=True))

print(new_train_ds)


In [ ]:
y_pred = np.argmax(model.predict(new_train_ds), axis=-1)

test_labels = test_labels.reshape((test_labels.size,))

print(y_pred, y_pred.size)
print(test_labels, test_labels.size)

In [ ]:
# hacky way of updating the size of the predictions set after the bug introduced by the batching.

predictions = np.zeros((test_labels.size,), dtype=int)

for i in range(y_pred.size):
  predictions[i] = y_pred[i]

print(y_pred, y_pred.size)
print(predictions, predictions.size)
print(test_labels, test_labels.size)

In [ ]:
# generating the confusion matrix 

con_mat = tf.math.confusion_matrix(labels=test_labels, predictions=predictions).numpy()

con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)

con_mat_df = pd.DataFrame(con_mat_norm,
                     index = CLASSES,
                     columns = CLASSES)

figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()